# Autoregressives Modell mit affinem Transformer auf MNIST

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
import os
import random

#Kleine Module von Lukas Rinder https://github.com/LukasRinder/normalizing-flows:
from LukasRinder.LukasRinder import load_and_preprocess_mnist
from LukasRinder.LukasRinder import Made
from LukasRinder.LukasRinder import train_density_estimation, nll

tfd = tfp.distributions
tfb = tfp.bijectors

tf.random.set_seed(1234)

### Nicht alle Versionen von Tensorflow-Probability sind mit alles Python Versionen kompatibel. Tfp steckt noch in der Beta-Version.

In [ ]:
print(np.__version__, tf.__version__, tfp.__version__, sep="\n")

### MNIST Daten von tf.keras.datasets laden, auf [0,1] skalieren und Batches initialisieren.

In [ ]:
batch_size = 128
category = -1
batched_train, batched_val, batched_test, _ = load_and_preprocess_mnist(
                                                    logit_space=False, batch_size=batch_size, classes=category)

In [ ]:
plt.imshow(next(iter(batched_train))[0], cmap='gray')

### Funktion, die ein MAF bzw. IAF Modell erzeugt.
#### Die Permutation ist hier fest gewählt und vertauscht die ersten 14 Zeilen mit den restlichen 14 als Block. Die einzelnen Transformationen werden mit tfb.Chain verkettet. Hier wird das in umgekehrter Reihenfolge getan, sodass die zuerst implementierte Transformation T1 entspricht (auf dem latenten Raum operiert).

In [ ]:
def AutoregressiveFlow(dimension, layers, hidden_shape=[512, 512], activation="relu", inverse=False):
    base_dist = tfd.MultivariateNormalDiag(loc=tf.zeros(shape=dimension, dtype=tf.float32))
    bijectors = []
    permutation = tf.cast(np.concatenate((np.arange(dimension/2,dimension),np.arange(0,dimension/2))), tf.int32)
    params=0
    if inverse:
        for i in range(layers):
            bijectors.append(tfb.Invert(tfb.MaskedAutoregressiveFlow(
                shift_and_log_scale_fn = Made(params=2, hidden_units=hidden_shape, activation=activation))))
            bijectors.append(tfb.Permute(permutation=permutation))
    else:
        for i in range(layers):
            bijectors.append(tfb.MaskedAutoregressiveFlow(
                shift_and_log_scale_fn = Made(params=2, hidden_units=hidden_shape, activation=activation)))
            bijectors.append(tfb.Permute(permutation=permutation))
        
    
    bijectors.append(tfb.Reshape(event_shape_out=(int(np.sqrt(dimension)),int(np.sqrt(dimension))),
                                 event_shape_in=(dimension,)))
    bijector = tfb.Chain(bijectors=list(reversed(bijectors)))
    
    masked_auto_flow = tfd.TransformedDistribution(distribution=base_dist, bijector=bijector)
    masked_auto_flow.log_prob(tf.reshape(base_dist.sample(), (28, 28)))
    for theta in masked_auto_flow.trainable_variables:
        params += np.prod(theta.shape)
    print("trainable parameters:", params)
    return masked_auto_flow, base_dist, bijectors, bijector

### Parameter festlegen und einen Namen für die Checkpoints festlegen.

In [ ]:
dataset = "mnist_all"
layers = 10
base_lr = 1e-3
end_lr = 1e-4
epochs = 80
mnist_trainsize = 50000
dimension = 784

### Modell initialisieren. In diesem Stadium entspricht MAF der Startverteilung bzw. full_bijector der Identitätsabbildung.

In [ ]:
MAF, base_dist, list_of_bijectors, full_bijector = AutoregressiveFlow(dimension, layers, inverse=False)

In [ ]:
learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(base_lr, epochs, end_lr, power=0.5)
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

### Checkpoints initialisieren.

In [ ]:
ckpt_dir = f"{dataset}/tmp_{layers}"
ckpt_prefix = os.path.join(ckpt_dir, "ckpt")

ckpt = tf.train.Checkpoint(optimizer=opt, model=MAF)

### Funktion, die ein Modell trainiert.
#### Dabei werden Trainings- und Validierungsdaten verwendet, um Overfitting festzustellen. Nach dem Durchlaufen aller Epochen, wird die benötigte Zeit ausgegeben.

In [ ]:
def TrainFlow(flow, batched_train, batched_val, epochs, train_size, optimizer, checkpoint, checkpoint_pref):

    t_losses, v_losses = [], []
    t_start = time.time()
    
    for i in range(epochs):
        batched_train.shuffle(buffer_size=train_size, reshuffle_each_iteration=True)
        batch_t_losses = []
        for batch in batched_train:
            batch_loss = train_density_estimation(flow, optimizer, batch)
            batch_t_losses.append(batch_loss)
        t_loss = tf.reduce_mean(batch_t_losses)

        batch_v_losses = []
        for batch in batched_val:
            batch_loss = nll(flow, batch)
            batch_v_losses.append(batch_loss)
        v_loss = tf.reduce_mean(batch_v_losses)

        t_losses.append(t_loss)
        v_losses.append(v_loss)
        print(f"Epoch {i+1}: train loss: {t_loss}, val loss: {v_loss}")
        
        if i == 0:
            min_v_loss = v_loss
            best_epoch = 0
        if v_loss < min_v_loss:
            min_v_loss = v_loss
            best_epoch = i
            checkpoint.write(file_prefix=checkpoint_pref)
                
    print("train time:", time.time() - t_start)
    
    return t_losses, v_losses

In [ ]:
train_losses, val_losses = TrainFlow(MAF, batched_train, batched_val, 
                                     epochs, mnist_trainsize, opt, ckpt, ckpt_prefix)

### Plot der Verluste während des Trainings.

In [ ]:
plt.plot(range(len(train_losses)), train_losses, label="train loss")
plt.plot(range(len(val_losses)), val_losses, label="val loss")
plt.legend()

### Laden des Stadiums des Modelles mit geringstem Verlust auf den Validierungsdaten.

In [ ]:
ckpt.restore(ckpt_prefix)

### Funktion, die den Hintergrund herausfiltert und die Helligkeit erhöht.

In [ ]:
def FilterBackroundPlot(sample, name="empty"):
    s = np.array(sample)
    s = s- np.median(s)
    s = np.abs(s)
    s = s/np.max(s)
    s = 255*s
    s = s.astype(int)
    s = np.reshape(s, 784)
    s = s*(3)
    s = np.where(s>255, 255, s)
    s = np.reshape(s, (28, 28))
    fig = plt.figure()
    plt.imshow(s, cmap="gray")
    if name != "empty":
        plt.savefig(name + ".png")

### Neue Daten generieren.

In [ ]:
n = 5
samples = MAF.sample(n)
for i in range(n):
    FilterBackroundPlot(samples[i])

### Funktion, die zweischen zwei Datenpunkten im latenten Raum linear Interpoliert.
#### Jedes 28x28 Pixel Bild kann genutzt werden. Je besser das Modell, desto realistischer sind die Zwischenschritte (bzw. desto weniger verblassen/erblassen die Datenpunkte einfach).

In [ ]:
def LatentInterpolation(start_point, end_point, bijector, epsilon=1/10, name="empty"):
    inverse = tfb.Invert(bijector)
    start = inverse.forward(start_point)
    end = inverse.forward(end_point)
    p = start
    plt.figure()
    FilterBackroundPlot(bijector.forward(p), name=name)
    for i in range(int(1/epsilon)):
        p += epsilon*(end-start)
        if name != "empty":
            name += str(i)
        FilterBackroundPlot(bijector.forward(p), name=name)

In [ ]:
real_start = next(iter(batched_train))[0]
real_end = next(iter(batched_train))[0]

LatentInterpolation(real_start, real_end, full_bijector)

### Veranschaulicht die Transformation von Rauschen zu Ziffer schrittweise. 
#### Alle 3 Schritte werden, durch die Permutation, die ersten 14 mit den letzten 14 Zeilen vertauscht.  Zwischenschritte mit vertauschten Blöcken nicht darzustellen würde aber auch autoregressive Schritte nicht zeigen.

In [ ]:
def FlowStepsMNIST(latent_point, bijectors_list, name="empty"):
    point = latent_point
    FilterBackroundPlot(point, name=name)
    counter = 1
    for bijector in bijectors_list:
        point = bijector.forward(point)
        if name != "empty":
            FilterBackroundPlot(point, name=name+str(counter))
        else:
            FilterBackroundPlot(point, name=name)
        counter += 1

In [ ]:
FlowStepsMNIST(full_bijector.inverse(next(iter(batched_train))[0]), list_of_bijectors)

### Vergleich der benötigten Zeit für die Vorwärts- bzw Rückwärtstransformation.
#### Hier muss full_bijector ineffizient genutzt werden (Vgl. nächste Zelle).

In [ ]:
latent = base_dist.sample(30)
real = next(iter(batched_train))[:30]

time_s = time.time()
for point in real:
    full_bijector.inverse(point)
time_e = time.time() - time_s
print("av_inverse_time:", time_e/30)

time_s = time.time()
for point in latent:
    full_bijector.forward(point)
time_e = time.time() - time_s
print("av_forward_time:", time_e/30)

### Durchschnittlich benötigte Zeit zum Generieren einer Stichprobe.

In [ ]:
time_s = time.time()
MAF.sample(50)
av_sample_time = (time.time() -time_s)/50
print(av_sample_time)